In [1]:
import ase
import ase.io
import jax
import jax.numpy as jnp
import jraph
import ml_collections
import numpy as np
import sys

sys.path.append('..')
from analyses import analysis
from symphony import train
from symphony.data import input_pipeline_tf

In [2]:
workdir = '/data/NFS/potato/songk/spherical-harmonic-net/workdirs/silica-mini-nequip-jan3'
rng = jax.random.PRNGKey(0)

In [3]:
config, best_state, best_eval_state, metrics_for_best_state = analysis.load_from_workdir(workdir)
config = ml_collections.FrozenConfigDict(config)
datasets = input_pipeline_tf.get_datasets(rng, config)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 13366359693578855492
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 13366359693578855492
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing 

In [4]:
metrics_for_best_state

{'train_eval': {'total_loss': array(6.242568, dtype=float32),
  'focus_and_atom_type_loss': array(2.9846961, dtype=float32),
  'position_loss': array(3.2578752, dtype=float32)},
 'val_eval': {'total_loss': array(6.3228116, dtype=float32),
  'focus_and_atom_type_loss': array(3.1943715, dtype=float32),
  'position_loss': array(3.128445, dtype=float32)},
 'test_eval': {'total_loss': array(6.6307435, dtype=float32),
  'focus_and_atom_type_loss': array(3.2746031, dtype=float32),
  'position_loss': array(3.3561542, dtype=float32)}}

In [5]:
frag_batch = next(datasets['train'].as_numpy_iterator())
unbatched_frags = jraph.unbatch(frag_batch)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 13366359693578855492
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 17912488874565519832
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing 

In [6]:
frag = unbatched_frags[1]

In [9]:
step_rng, rng = jax.random.split(rng)
step_rngs = jax.random.split(step_rng, jax.local_device_count())
preds = train.get_predictions(best_eval_state, frag, rng)

In [7]:
frag_ase = ase.Atoms(
    positions = frag.nodes.positions,
    numbers = np.vectorize(lambda x: 14 if x else 8)(frag.nodes.species),
    cell = frag.globals.cell[0],
    pbc = np.array([1, 1, 1]),
)
# ase.io.write('frag.cif', frag_ase)

In [8]:
frag.nodes.focus_and_target_species_probs

Array([[0.6666667 , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.

In [50]:
frag.receivers[frag.senders == 3]

Array([29], dtype=int32)

In [51]:
frag.nodes.species[3]

Array(1, dtype=int32)

In [12]:
preds.nodes.focus_and_target_species_probs  # what's going on here

Array([[2.25300685e-01, 7.47848753e-06],
       [1.64458600e-10, 4.10926230e-13],
       [6.42328551e-12, 1.28326228e-14],
       [6.27623945e-02, 2.07922153e-06],
       [2.62741387e-01, 1.24207861e-06],
       [7.31344149e-02, 3.62813967e-06],
       [8.13686114e-04, 8.96174527e-07],
       [2.77690440e-02, 5.45509392e-05],
       [8.64883259e-05, 5.09033489e-05],
       [8.55976978e-05, 7.03540863e-05],
       [1.25342041e-07, 1.58928509e-10],
       [5.06151707e-07, 2.35612030e-10],
       [1.17482744e-08, 1.01320142e-11],
       [9.16255694e-09, 1.24159225e-11],
       [3.79054001e-07, 1.05017128e-09],
       [8.95798507e-07, 1.68308070e-08],
       [5.76642663e-07, 2.79035306e-09],
       [3.17346377e-07, 5.73704195e-09],
       [9.50147587e-05, 6.50118163e-04],
       [3.20274616e-04, 1.19150837e-03],
       [6.53811439e-05, 2.46945373e-03],
       [7.59533359e-05, 2.17542704e-03],
       [2.97963183e-04, 1.20822375e-03],
       [4.85993311e-04, 3.14384699e-04],
       [2.120037